<a href="https://colab.research.google.com/github/abhinavarorags/CoolStuff/blob/test/QuantHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
print("stuff")

stuff


In [17]:
import pandas as pd
url = 'https://raw.githubusercontent.com/abhinavarorags/CoolStuff/refs/heads/test/s.csv'
url = 'https://raw.githubusercontent.com/abhinavarorags/CoolStuff/refs/heads/test/sample_data.csv'

try:
  df = pd.read_csv(url, sep=',')
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")
df.head()
print("Data loaded: ",f"Len: {len(df)}, Shape: {df.shape[0]}, Index Size: {df.index.size}, Size/Columns: {df.size // df.columns.size}")


Data loaded:  Len: 67048, Shape: 67048, Index Size: 67048, Size/Columns: 67048


In [18]:
df_training = df.iloc[:40000]  # First 100 rows
train_data = df_training
df_test = df.iloc[40001:]     # Rows from index 100 onwards
test_data = df_test.copy()

In [ ]:
#DO NOT RUN THIS CELL
#This is the REAL Q-learning code but it runs slower
#Also needs lots of GPU power
#See image asking for 84 hours of runtime
#https://github.com/abhinavarorags/CoolStuff/blob/test/QuantFury%20Q-learning%20Compute.png

def foo():
  import numpy as np
  import pandas as pd
  import random

  # Using the previously loaded train_data from '/mnt/data/training.csv'

  # Q-learning parameters
  initial_alpha = 0.5  # Initial learning rate
  min_alpha = 0.01  # Minimum learning rate
  alpha_decay = 0.995  # Decay rate for alpha

  initial_epsilon = 1.0  # Initial exploration rate
  min_epsilon = 0.01  # Minimum exploration rate
  epsilon_decay = 0.995  # Decay rate for epsilon

  gamma = 0.95  # Discount factor
  num_episodes = 1000  # Number of episodes

  # Define actions
  ACTIONS = {0: 'Buy Long', 1: 'Sell Short', 2: 'Hold'}

  # Initialize Q-table
  q_table = np.zeros((len(train_data), len(ACTIONS)))

  # Function to choose an action using epsilon-greedy strategy
  def choose_action(state_index, epsilon):
      if random.uniform(0, 1) < epsilon:
          return random.choice(list(ACTIONS.keys()))  # Explore
      else:
          return np.argmax(q_table[state_index])  # Exploit

  # Q-learning process
  for episode in range(num_episodes):
      alpha = max(min_alpha, initial_alpha * (alpha_decay ** episode))
      epsilon = max(min_epsilon, initial_epsilon * (epsilon_decay ** episode))
      total_reward = 0

      for state_index in range(len(train_data) - 1):
          current_state = train_data.iloc[state_index].values[:-1]  # Exclude Date
          action = choose_action(state_index, epsilon)
          reward = train_data.iloc[state_index + 1]['Close'] - train_data.iloc[state_index]['Close']
          next_state_index = state_index + 1
          best_future_q = np.max(q_table[next_state_index]) if next_state_index < len(train_data) else 0
          q_table[state_index, action] += alpha * (reward + gamma * best_future_q - q_table[state_index, action])
          total_reward += reward

      if episode % 100 == 0:
          print(f"Episode: {episode}, Alpha: {alpha:.4f}, Epsilon: {epsilon:.4f}, Total Reward: {total_reward}")

  # Save the Q-table to CSV
  q_table_df = pd.DataFrame(q_table, columns=ACTIONS.keys())
  #q_table_df.to_csv('/mnt/data/q_table.csv', index=False)
  print("Q-table training complete and saved as q_table.csv.")

if 1==1:
  pass
else:
    foo()


In [29]:
#Dumbed down the code to only 'Close' Prices and 100 episodes
print("Running Code on Training Data")

import numpy as np
import pandas as pd
import random

# Using the previously loaded train_data from '/mnt/data/training.csv'

# Q-learning parameters
initial_alpha = 0.5  # Initial learning rate
min_alpha = 0.01  # Minimum learning rate
alpha_decay = 0.995  # Decay rate for alpha

initial_epsilon = 1.0  # Initial exploration rate
min_epsilon = 0.01  # Minimum exploration rate
epsilon_decay = 0.995  # Decay rate for epsilon

gamma = 0.95  # Discount factor
# Reduce the number of episodes for demonstration
num_episodes = 100  # Use a smaller number for quicker demonstration

ACTIONS = {0: 'Buy Long', 1: 'Sell Short', 2: 'Hold'}

# Convert necessary columns to NumPy arrays for faster access
close_prices = train_data['Close'].values
actions = np.array(list(ACTIONS.keys()))



# Initialize Q-table
q_table = np.zeros((len(close_prices) - 1, len(actions)))

# Run the Q-learning algorithm with optimized data handling
for episode in range(num_episodes):
    alpha = max(min_alpha, initial_alpha * (alpha_decay ** episode))
    epsilon = max(min_epsilon, initial_epsilon * (epsilon_decay ** episode))
    total_reward = 0

    for i in range(len(close_prices) - 1):
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)
        else:
            action = actions[np.argmax(q_table[i])]

        reward = close_prices[i + 1] - close_prices[i]
        next_state_action_values = q_table[i + 1] if i + 1 < len(close_prices) - 1 else np.zeros(len(actions))
        best_future_q = np.max(next_state_action_values)
        q_table[i, action] += alpha * (reward + gamma * best_future_q - q_table[i, action])
        total_reward += reward

    print(f"Episode: {episode}, Alpha: {alpha:.4f}, Epsilon: {epsilon:.4f}, Total Reward: {total_reward}")
    # Print the Q-table after each episode to see if it's updating
    if episode % 1 == 0:  # Modify to control how often you see the output
        print(f"After Episode {episode}: Q-Table Sample:")
        print(q_table[:5])  # Print first few rows of the Q-table to check updates


# Convert Q-table to DataFrame and save to CSV
q_table_df = pd.DataFrame(q_table, columns=[str(action) for action in actions])
#q_table_df.to_csv('/mnt/data/q_table_optimized.csv', index=False)
print("Optimized Q-table training complete and saved.")


Running Code on Training Data
Episode: 0, Alpha: 0.5000, Epsilon: 1.0000, Total Reward: -6.469999999999999
After Episode 0: Q-Table Sample:
[[ 0.     0.3    0.   ]
 [ 0.     0.    -0.265]
 [-0.16   0.     0.   ]
 [ 0.     0.475  0.   ]
 [ 0.    -0.24   0.   ]]
Episode: 1, Alpha: 0.4975, Epsilon: 0.9950, Total Reward: -6.469999999999999
After Episode 1: Q-Table Sample:
[[ 0.          0.44925     0.        ]
 [ 0.         -0.263675   -0.265     ]
 [-0.16        0.06529688  0.        ]
 [ 0.          0.7113125   0.        ]
 [ 0.         -0.3594      0.        ]]
Episode: 2, Alpha: 0.4950, Epsilon: 0.9900, Total Reward: -6.469999999999999
After Episode 2: Q-Table Sample:
[[ 0.          0.52387313  0.        ]
 [-0.23164999 -0.263675   -0.265     ]
 [-0.16        0.06529688  0.17609915]
 [ 0.          0.8294658   0.        ]
 [-0.237606   -0.3594      0.        ]]
Episode: 3, Alpha: 0.4925, Epsilon: 0.9851, Total Reward: -6.469999999999999
After Episode 3: Q-Table Sample:
[[ 0.          0.

KeyboardInterrupt: 

In [25]:
q_table=q_table_df #Reassigned from pandas data frame to numpy array Hack
test_data_orig = test_data.copy()
print("Running Code on Test Data")

# Extracting Close prices from the test data for action determination
close_prices_test = test_data['Close'].values

# Preparing the output DataFrame
recommended_actions = []

# Determine the best action for each state based on the Q-table
for i in range(len(close_prices_test) - 1):
    state_index = i % len(q_table)  # Looping through Q-table if test data is longer than Q-table
    action_index = np.argmax(q_table.iloc[state_index])
    recommended_actions.append(ACTIONS[action_index])

# Adding recommended actions to the test data
test_data['Recommended Action'] = recommended_actions + ['N/A']  # Last entry has no subsequent state

# Save the results with recommendations
#test_data.to_csv('/mnt/data/test_with_actions.csv', index=False)
print("Test Data saved with recommendations")


Running Code on Test Data
Test Data saved with recommendations


In [26]:
#test_data.iloc[4:5]
test_with_actions = test_data
test_data = test_data_orig
print("Test data with recommendations loaded: ",f"Len: {len(test_with_actions)}, Shape: {test_with_actions.shape[0]}, Index Size: {test_with_actions.index.size}, Size/Columns: {test_with_actions.size // test_with_actions.columns.size}")

Test data with recommendations loaded:  Len: 27047, Shape: 27047, Index Size: 27047, Size/Columns: 27047


In [27]:
# Load necessary libraries
import pandas as pd

# Assuming you have loaded 'test_with_actions' as follows
#test_with_actions = pd.read_csv('path_to_test_with_actions.csv')

# Define threshold for 'Hold' minimal change
threshold = 0.005

# Calculate correct and wrong predictions for 'Buy Long'
correct_buy_long = test_with_actions[(test_with_actions['Recommended Action'] == 'Buy Long') & (test_with_actions['Close'] > test_with_actions['Open'])]
wrong_buy_long = test_with_actions[(test_with_actions['Recommended Action'] == 'Buy Long') & (test_with_actions['Close'] <= test_with_actions['Open'])]

# Calculate correct and wrong predictions for 'Sell Short'
correct_sell_short = test_with_actions[(test_with_actions['Recommended Action'] == 'Sell Short') & (test_with_actions['Close'] < test_with_actions['Open'])]
wrong_sell_short = test_with_actions[(test_with_actions['Recommended Action'] == 'Sell Short') & (test_with_actions['Close'] >= test_with_actions['Open'])]

# Calculate correct and wrong predictions for 'Hold'
correct_hold = test_with_actions[(test_with_actions['Recommended Action'] == 'Hold') & (abs(test_with_actions['Close'] - test_with_actions['Open']) / test_with_actions['Open'] < threshold)]
wrong_hold = test_with_actions[(test_with_actions['Recommended Action'] == 'Hold') & (abs(test_with_actions['Close'] - test_with_actions['Open']) / test_with_actions['Open'] >= threshold)]

# Compile results into a DataFrame
results = {
    "Buy Long": {"Correct": len(correct_buy_long), "Wrong": len(wrong_buy_long)},
    "Sell Short": {"Correct": len(correct_sell_short), "Wrong": len(wrong_sell_short)},
    "Hold": {"Correct": len(correct_hold), "Wrong": len(wrong_hold)}
}

results_df = pd.DataFrame(results).transpose()
results_df.columns = ['Correct Predictions', 'Wrong Predictions']

# Display the results DataFrame
print(results_df)


            Correct Predictions  Wrong Predictions
Buy Long                   4555               4559
Sell Short                 4393               4503
Hold                       7395               1607


In [31]:
import numpy as np
import pandas as pd
import random

def load_data(url):
    # Load the training data from URL
    try:
        df = pd.read_csv(url, sep=',')
    except pd.errors.ParserError as e:
        print(f"Error reading CSV: {e}")
        raise

    df_training = df.iloc[:20000]  # First 20000 rows
    df_test = df.iloc[20001:]  # Rows from index 20001 onwards
    return df_training, df_test

def train_q_learning(train_data, num_episodes=200):
    # Q-learning parameters
    initial_alpha = 0.5  # Initial learning rate
    min_alpha = 0.01  # Minimum learning rate
    alpha_decay = 0.995  # Decay rate for alpha

    initial_epsilon = 1.0  # Initial exploration rate
    min_epsilon = 0.01  # Minimum exploration rate
    epsilon_decay = 0.995  # Decay rate for epsilon

    gamma = 0.95  # Discount factor
    ACTIONS = {0: 'Buy Long', 1: 'Sell Short', 2: 'Hold'}

    # Convert necessary columns to NumPy arrays for faster access
    close_prices = train_data['Close'].values
    actions = np.array(list(ACTIONS.keys()))

    # Initialize Q-table
    q_table = np.zeros((len(close_prices) - 1, len(actions)))

    # Run the Q-learning algorithm with optimized data handling
    for episode in range(num_episodes):
        alpha = max(min_alpha, initial_alpha * (alpha_decay ** episode))
        epsilon = max(min_epsilon, initial_epsilon * (epsilon_decay ** episode))
        total_reward = 0

        for i in range(len(close_prices) - 1):
            if random.uniform(0, 1) < epsilon:
                action = random.choice(actions)
            else:
                action = actions[np.argmax(q_table[i])]

            reward = close_prices[i + 1] - close_prices[i]
            next_state_action_values = q_table[i + 1] if i + 1 < len(close_prices) - 1 else np.zeros(len(actions))
            best_future_q = np.max(next_state_action_values)
            q_table[i, action] += alpha * (reward + gamma * best_future_q - q_table[i, action])
            total_reward += reward

        print(f"Episode: {episode}, Alpha: {alpha:.4f}, Epsilon: {epsilon:.4f}, Total Reward: {total_reward}")
        # Print the Q-table after each episode to see if it's updating
        if episode % 1 == 0:  # Modify to control how often you see the output
            print(f"After Episode {episode}: Q-Table Sample:")
            print(q_table[:5])  # Print first few rows of the Q-table to check updates

    return q_table

def test_q_learning(test_data, q_table):
    print("Running Code on Test Data")

    ACTIONS = {0: 'Buy Long', 1: 'Sell Short', 2: 'Hold'}
    # Extracting Close prices from the test data for action determination
    close_prices_test = test_data['Close'].values

    # Preparing the output DataFrame
    recommended_actions = []

    # Determine the best action for each state based on the Q-table
    for i in range(len(close_prices_test) - 1):
        state_index = i % len(q_table)  # Looping through Q-table if test data is longer than Q-table
        action_index = np.argmax(q_table[state_index])
        recommended_actions.append(ACTIONS[action_index])

    # Adding recommended actions to the test data
    test_data['Recommended Action'] = recommended_actions + ['N/A']  # Last entry has no subsequent state

    # Save the results with recommendations
    # test_data.to_csv('/mnt/data/test_with_actions.csv', index=False)
    print("Test Data saved with recommendations")
    return test_data

def performanceAnalyse(test_data_with_actions):
    # Assuming you have loaded 'test_with_actions' as follows
    #test_with_actions = pd.read_csv('path_to_test_with_actions.csv')

    # Define threshold for 'Hold' minimal change
    threshold = 0.005

    # Calculate correct and wrong predictions for 'Buy Long'
    correct_buy_long = test_with_actions[(test_with_actions['Recommended Action'] == 'Buy Long') & (test_with_actions['Close'] > test_with_actions['Open'])]
    wrong_buy_long = test_with_actions[(test_with_actions['Recommended Action'] == 'Buy Long') & (test_with_actions['Close'] <= test_with_actions['Open'])]

    # Calculate correct and wrong predictions for 'Sell Short'
    correct_sell_short = test_with_actions[(test_with_actions['Recommended Action'] == 'Sell Short') & (test_with_actions['Close'] < test_with_actions['Open'])]
    wrong_sell_short = test_with_actions[(test_with_actions['Recommended Action'] == 'Sell Short') & (test_with_actions['Close'] >= test_with_actions['Open'])]

    # Calculate correct and wrong predictions for 'Hold'
    correct_hold = test_with_actions[(test_with_actions['Recommended Action'] == 'Hold') & (abs(test_with_actions['Close'] - test_with_actions['Open']) / test_with_actions['Open'] < threshold)]
    wrong_hold = test_with_actions[(test_with_actions['Recommended Action'] == 'Hold') & (abs(test_with_actions['Close'] - test_with_actions['Open']) / test_with_actions['Open'] >= threshold)]

    # Compile results into a DataFrame
    results = {
        "Buy Long": {"Correct": len(correct_buy_long), "Wrong": len(wrong_buy_long)},
        "Sell Short": {"Correct": len(correct_sell_short), "Wrong": len(wrong_sell_short)},
        "Hold": {"Correct": len(correct_hold), "Wrong": len(wrong_hold)}
    }

    results_df = pd.DataFrame(results).transpose()
    results_df.columns = ['Correct Predictions', 'Wrong Predictions']

    # Display the results DataFrame
    print(results_df)

# Main script
url = 'https://raw.githubusercontent.com/abhinavarorags/CoolStuff/refs/heads/test/sample_data.csv'
train_data, test_data = load_data(url)
q_table = train_q_learning(train_data)

test_with_actions = test_q_learning(test_data, q_table)

# Analyze the performance
performanceAnalyse(test_with_actions)


Episode: 0, Alpha: 0.5000, Epsilon: 1.0000, Total Reward: 173.41
After Episode 0: Q-Table Sample:
[[ 0.3    0.     0.   ]
 [ 0.    -0.265  0.   ]
 [ 0.     0.    -0.16 ]
 [ 0.     0.     0.475]
 [ 0.     0.    -0.24 ]]
Episode: 1, Alpha: 0.4975, Epsilon: 0.9950, Total Reward: 173.41
After Episode 1: Q-Table Sample:
[[ 0.44925     0.          0.        ]
 [ 0.         -0.265      -0.263675  ]
 [ 0.          0.         -0.01510312]
 [ 0.472625    0.          0.475     ]
 [ 0.          0.         -0.3594    ]]
Episode: 2, Alpha: 0.4950, Epsilon: 0.9900, Total Reward: 173.41
After Episode 2: Q-Table Sample:
[[ 0.44925     0.2970075   0.        ]
 [-0.26235663 -0.265      -0.263675  ]
 [ 0.          0.06497039 -0.01510312]
 [ 0.472625    0.47026188  0.475     ]
 [ 0.         -0.237606   -0.3594    ]]
Episode: 3, Alpha: 0.4925, Epsilon: 0.9851, Total Reward: 173.41
After Episode 3: Q-Table Sample:
[[ 0.44925     0.32348321  0.        ]
 [-0.36378067 -0.265      -0.263675  ]
 [ 0.06464554  0.